# **SBERT(Sentence Transformers)**

# **Settings**

In [ ]:
! pip install koreanize_matplotlib

In [ ]:
! pip install sentence_transformers

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')                       # warning 출력 false

import pandas as pd
import numpy as np
np.set_printoptions(precision=6, suppress=True)     # 소수점 6자리까지만 표현해준다.
import random

import matplotlib.pyplot as plt
import koreanize_matplotlib

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# **1. 모델 불러오기**

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
sentence = ['안녕하세요 반갑습니다.','임베딩이 잘될까요?']
embeding = model.encode(sentence)
embeding.shape

(2, 768)

In [ ]:
print(embeding)

[[-0.415056  0.114084  0.988263 ...  0.361861  0.02847   0.308759]
 [ 0.111308 -0.23882   1.320755 ...  0.052853 -0.272375  0.045653]]


# **2. 데이터 다운받기**


In [ ]:
!wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv

--2023-07-11 06:41:38--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv’

ChatbotData.csv     100%[===================>] 868.99K  --.-KB/s    in 0.02s   

2023-07-11 06:41:38 (49.7 MB/s) - ‘ChatbotData.csv’ saved [889842/889842]



In [ ]:
data = pd.read_csv('./ChatbotData.csv')
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [ ]:
import csv

with open('./embeding.csv', 'w') as f:
    mapdata = []
    write = csv.writer(f)

    for temp in data['Q']:
        emb = model.encode(temp)
        mapdata.append(emb)

    write.writerows(mapdata)


In [ ]:
data1 = pd.read_csv('./embeding.csv',header=None)
data1.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.201796,-0.034438,1.539572,0.010697,0.379792,0.142493,0.182864,0.558312,-0.394889,-0.117658,...,0.153538,-0.053811,0.381607,-0.133929,-0.189685,-0.460535,-0.108984,0.059759,0.120592,0.119612
1,0.077166,-0.034278,0.862442,0.026361,0.209838,0.113579,0.339394,0.206503,-0.070944,-0.153693,...,-0.137552,-0.215384,0.240824,-0.393945,-0.199295,-0.225587,-0.097706,0.073397,-0.243929,0.068427
2,0.104452,-0.012432,1.013288,0.022502,0.151803,0.076846,0.277340,0.083339,-0.039824,-0.118836,...,-0.304560,-0.371909,0.133966,-0.416744,-0.214807,-0.151465,-0.024058,0.040367,-0.208522,0.114699
3,0.097607,-0.046717,0.893694,0.021047,0.169932,0.062547,0.262190,0.059330,-0.009971,-0.139920,...,-0.313720,-0.413321,0.109724,-0.433837,-0.220793,-0.122533,-0.026790,0.045947,-0.215361,0.107051
4,-0.070029,0.031961,1.491543,0.000043,0.186841,0.102466,0.271964,0.463555,-0.338619,-0.016967,...,0.052661,-0.331358,0.326646,-0.443471,-0.241182,-0.264238,-0.167959,0.273983,-0.174544,0.311920


In [ ]:
def chatbot_text(text):
    emb_result = model.encode(text)

    cosine_result = []
    for temp in range(len(data1)):
        data_temp = data1.iloc[temp]
        cosine_result.append(cosine_similarity([data_temp],[emb_result])[0][0])

    data['cosine'] = cosine_result
    data_result = data.sort_values('cosine',ascending=False)

    r = random.randint(0,5)

    return data_result.iloc[r]

In [ ]:
chatbot_text('오늘 날씨 진짜 구리다')

Q         난방비 비싼데 추워
A          따뜻하게 사세요!
label              0
cosine      0.758436
Name: 687, dtype: object

In [ ]:
# elastic search 이용하면 응답시간 더 줄어든다.